In [1]:

import re
import json

class ParseError(Exception):
    pass

class Parser:
    def __init__(self):
        pass

    def remove_prefix(self, entity):
        """Loại bỏ các prefix như 'wdt:' và 'wd:'"""
        return entity.split(':')[-1] if ':' in entity else entity

    def triplet_to_clause(self, tgt_var, triplet, parsed_dict):
        """Chuyển đổi một bộ ba thành câu lệnh S-Expression, loại bỏ prefix"""
        if triplet[0] == tgt_var:
            other = self.remove_prefix(triplet[-1])
            if other in parsed_dict:
                other = '( ' + parsed_dict[other] + ' )'
            return 'JOIN {} {}'.format(self.remove_prefix(triplet[1]), other)
        elif triplet[-1] == tgt_var:
            other = self.remove_prefix(triplet[0])
            if other in parsed_dict:
                other = '( ' + parsed_dict[other] + ' )'
            return 'JOIN ( R {} ) {}'.format(self.remove_prefix(triplet[1]), other)
        else:
            raise ParseError()

    def build_dependency_graph(self, triples, target_variable):
        """Xây dựng đồ thị phụ thuộc từ danh sách bộ ba và biến mục tiêu."""
        dependency_list = []
        related_triples = [triple for triple in triples if target_variable in triple]
        if related_triples:
            dependency_list.append((target_variable, related_triples))

        new_target_variables = set()
        for triple in related_triples:
            for variable in triple:
                if variable != target_variable and variable.startswith('?'):
                    new_target_variables.add(variable)

        for new_target in new_target_variables:
            related_triples_for_new_target = [triple for triple in triples if new_target in triple]
            related_triples_for_new_target = [triple for triple in related_triples_for_new_target if triple not in related_triples]
            if related_triples_for_new_target:
                dependency_list.append((new_target, related_triples_for_new_target))

        return dependency_list

    def dep_graph_to_s_expr(self, var_dep_list, ret_var, spec_condition=None):
        if not var_dep_list:
            raise ParseError("Danh sách phụ thuộc không có phần tử nào.")
        if var_dep_list[0][0] != ret_var:
            raise ParseError("Biến trả về không khớp với biến đầu tiên trong danh sách phụ thuộc.")

        var_dep_list.reverse()
        parsed_dict = {}

        for var_name, dep_relations in var_dep_list:
            clause = self.triplet_to_clause(var_name, dep_relations[0], parsed_dict)
            for tri in dep_relations[1:]:
                n_clause = self.triplet_to_clause(var_name, tri, parsed_dict)
                clause = 'AND ( {} ) ( {} )'.format(n_clause, clause)
            parsed_dict[var_name] = clause

        res = '( ' + parsed_dict[ret_var] + ' )'
        return res


In [2]:

# Đọc dữ liệu từ file input.json
# with open('LC-QuAD2.0/filter/test.json', 'r', encoding='utf-8') as f:
#     data = json.load(f)

with open('LC-QuAD2.0/filter/train.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Tạo danh sách mới để lưu trữ dữ liệu với các thuộc tính mới
new_data = []

# Khởi tạo parser
parser = Parser()

# Xử lý từng điểm dữ liệu
i = 0
for entry in data:
    sparql_query = entry['sparql_wikidata']
    
    # Tìm biến mục tiêu
    match = re.search(r'select(?: distinct)? (\?[\w]+) (?:where)', sparql_query, re.IGNORECASE)
    if match:
        target_variable = match.group(1)  # Lưu biến mục tiêu

        # Tách các bộ ba theo dấu "."
        triples = re.findall(r'\{(.*?)\}', sparql_query)
        all_triples = []  # Danh sách để lưu tất cả các bộ ba cho entry này

        for triple in triples:
            # Tách các bộ ba riêng biệt
            individual_triples = [t.strip() for t in triple.split('.') if t.strip()]
            for individual_triple in individual_triples:
                # Tách các thành phần trong từng bộ ba theo khoảng trắng
                components = [component.strip() for component in individual_triple.split() if component]
                # Chỉ thêm vào nếu có đúng 3 thành phần
                if len(components) == 3:
                    all_triples.append(components)  # Thêm bộ ba vào danh sách

        # Chuyển đổi danh sách bộ ba sang cấu trúc S-Expression
        dependency_graph = parser.build_dependency_graph(all_triples, target_variable)
        s_expression = parser.dep_graph_to_s_expr(dependency_graph, target_variable)

        # Lưu vào new_data chỉ một lần cho mỗi target_variable
        new_data.append({
            "question_id": i, 
            'question': entry['question'],
            "question_vi":entry['question_vi'],
            'sparql': entry['sparql_wikidata'],
            'target_variable': target_variable,
            'triples': all_triples,  # Lưu tất cả bộ ba trong danh sách
            'answer': entry['answer'],
            's_expr': s_expression  # Lưu S-Expression
        })
        i = i + 1

# Lưu kết quả vào file input_train.json
# with open('LC-QuAD2.0/s_expr/test.json', 'w', encoding='utf-8') as f:
#     json.dump(new_data, f, ensure_ascii=False, indent=4)

In [3]:
with open('LC-QuAD2.0/s_expr/train.json', 'w', encoding='utf-8') as f:
    json.dump(new_data, f, ensure_ascii=False, indent=4)